# Mini Project Notebook

## INTRODUCTION

### Structure of the team

The group members for this project are Martin Alonso, Sena Beste Ercan, Dara El-Shaarawi and Katherine Robles. Instead of assigning roles for the four of us, we have each worked on every component of the project leading up to the final submission. Creating the environment, writing the notebook and coding the robots behaviors were all equally divided into four so that each group member could contribute. We all have basic or minimal knowledge of Python, allowing us to build off of each other's ideas rather than work independently.

### Scenario and scientific literature

We wanted to create a scenario in which **epuck2**, the *thief*, is trying to reach the food source, and **epuck1**, the *police*, tries to prevent this crime from taking place. When the police realizes that the thief is going to steal food from the source, it starts to chase the thief, and the thief starts to run away from the police until it reaches its safe space. In scientific literature, these behaviors can be attributed to the fight-or-flight responses as well as predator-prey behavior.

Most species are a part of predator-prey relationships, whether that is by searching for their own sustenance or being victim to it from another (1). Predator-prey behaviors are a foundation of many environments (1) and ignite the fight or flight response in situations of threat. For that reason, understanding these connections offers an approach to replicate these complex behaviors. This has been done in prior robotics research in which robot predators are driven to capture robot prey (2) and their instincts are even leveraged as a mechanism to maintain robots safety (3). Our objective was to simulate these natural behaviors that have been seen in animals and replicated in robotics in a simplistic yet fruitful way. To do so, we have created a scenario that is based off of a ‘human’ interaction that still showcases simple instinctual behaviors found in nature.

### Details of the environment and the cognitive modelling

In the environment, there are two ePucks. One of them represents the police, and the other one represents the thief. Spheres are the food source that the thief will begin to acquire as it travels across the environment. These spheres will serve as energy boosters that increase the thief’s energy and encourage it to run away from the police. The cups and trees serve as a safe place for the thief, in which the police cannot enter.
When the energy level is lower, behavior of escaping is weighted so it does not escape 

## IMPLEMENTATION

When the thief is spotted by the police, it will chase the thief. Upon this interaction, the thief will run away from the police. To allow the robots to interact with the environment correctly, obstacle avoidance is attached to both robots, leading them to avoid the walls. However, the ePuck that is playing the role of the police is attached with an obstacle avoidance behavior that includes the cups and trees in the environment. Doing so allows for the thief to run away from the police successfully into an area that the police cannot enter.

The first step is to connect this Jupyter Notebook to the simulator exceuting the code below:

In [ ]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2 = open_session(n_epucks=2)

The next step is to define all the behaviors and routines:

In [ ]:
# Defining obstacle_avoidance behavior to avoid the walls
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

# Defining the behavior to avoid to go to the safe place
def safe_place_avoidance(robot): 
    left, right = robot.prox_activations(tracked_objects=["Cup", "Tree"])
    left_wheel = left
    right_wheel = right
    return left_wheel,  right_wheel, robot.energy_level

# Defining the behavior to go to the safe place
def towards_safe_place(robot): 
    left, right = robot.prox_activations(tracked_objects=["Tree"])
    left_wheel = 1 - left
    right_wheel = 1 - right
    return left_wheel, right_wheel, robot.energy_level

The cell below corresponds to the obstacles perceived by the left and right proximeters of the epuck1. The higher the value, the closer the object.

In [ ]:
# Storing the values of the left and right proximeters... 
left, right = epuck1.prox_activations(tracked_objects=["ePuck"])
print(left, right)

We need spheres appear in the environment:

In [ ]:
# Start sphere apparition in the environment
simulator.start_sphere_apparition(period=20., min_pos=[-1.3, 0.66, 0], max_pos=[-1.1, 0.68, 1])

Now, once spheres are in the environment, other behaviors and routines are defined according to that:

In [ ]:
# Defining foraging behavior to catch food sources
def foraging (robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    return right, left

# Defining routine in which the ePucks increases/decreases its energy level according to the spheres eaten or not
def energy_drive (robot): 
    if robot.has_eaten():
        robot.energy_level += 0.2  # if the robot has eaten a sphere, increase its energy level by 0.2
    else:
        robot.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    robot.energy_level = min(1., max(robot.energy_level, 0.))

# It is necessary to define an initial level of danger of 0 in both of the ePucks
epuck1.danger = 0
epuck2.danger = 0
# Defining a routine to react according to the level of danger
def alarm_drive (robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    if left + right > 0:
        robot.danger = left + right 
    else:
        robot.danger = 0
    # The line below bounds the value of the robot's danger between 0 and 1
    robot.danger = min(1., max(robot.danger, 0.))

# Defining a routine to record energy data
def epuck_log(robot):
    # Retrieving the values of the left and right proximeters:
    left, right = robot.prox_activations()
    # Recording the energy level in the topic called "energy"
    robot.add_log("energy", robot.energy_level)

# Defining the chasing behavior for the police
def chasing (robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    return right, left, robot.energy_level

# Defining the running away behavior for the thief
def escaping (robot):
    if robot.danger > 0:
        left, right = robot.prox_activations(tracked_objects=["ePuck"])
        return left, right
    else:
        left, right = robot.prox_activations(tracked_objects=["Cup"])
        return right, left, robot.energy_level

Then, the behaviors and routines defined above have to be attached to the ePucks:

In [ ]:
for e in simulator.robots:
    # First, detach all behaviors in case one is attached to any ePuck
    e.detach_all_behaviors()
    
    # Attaching obstacle_avoidance and foraging behaviors
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.attach_behavior(foraging, freq=10)
    
    # Attaching alarm_drive, energy_drive and epuck_log routines
    e.attach_routine(alarm_drive, freq=1)
    e.attach_routine(energy_drive, freq=1)
    e.attach_routine(epuck_log, freq=1)
    
    # It is necessary to define an initial level of energy before staring routines
    e.energy_level = 0.5

# ePuck1 = Police. Attaching corresponding behaviors
epuck1.attach_behavior(chasing, freq=10)
epuck1.attach_behavior(safe_place_avoidance, freq=10)

# ePuck2 = Thief. Attaching corresponding behaviors
epuck2.attach_behavior(escaping, freq=10)
epuck2.attach_behavior(towards_safe_place, freq=10)

# Starting all behaviors and routines attached to the ePucks
for e in simulator.robots:
    e.start_all_behaviors()
    e.start_all_routines()

Printing the energy levels of both of the ePucks: 

In [ ]:
print(epuck1.energy_level)
print(epuck2.energy_level)

With the code below, the spheres apparition is stopped:

In [ ]:
simulator.stop_sphere_apparition()

Execute the cell below to stop all the behaviors and routines attached to the ePucks:

In [ ]:
for e in simulator.robots:
    e.stop_all_behaviors()
    e.stop_all_routines()
    e.detach_all_behaviors()
    e.detach_all_routines()
    e.stop()

Close the simulator session:

In [ ]:
close_session(simulator)

## RESULTS

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%pylab inline

# Plot the energy levels recorded by `epuck1` and `epuck2`
plot(epuck1.get_log("energy"))
plot(epuck2.get_log("energy"))
legend(["Police", "Thief"])

xlabel("Time")
ylabel("Energy level")
title("Plot of energy level against time")

Put description of the results here...

## DISCUSSION 

### Summary 

Overall, the project went smoothly and we were able to succeed in programming the ePucks within our environment to act accordingly. The work accomplished by our team was overall very organized and robust. Once our scenario was created, we began with our environmental design, then proceeded to write and test our code, plot the graphs, then assemble our completed Jupyter Notebook. From the plots, we were able to see….

### Limitations 

There are issues that were encountered during the implementation process. In terms of the code structure, we encountered errors that led us to simplify our code in order to allow it to perform accordingly and run seamlessly. Additionally, some obstacles were encountered in relation to our ePucks behaviors. For example, the behaviors in the robots were only activated if they were in close proximity to one another, leading us to believe that the code was not working. However, this behavior mimicked natural behaviors in the sense that the prey will only chase the predator if it is sensed within the environment. Another limitation in our method was that there was minimal interaction between the thief, police, and the objects around them. For example, the thief was not being able to take what has been stolen to its home but rather the interaction was limited to the thief eating spheres as a representation of a stealing behavior.

### Possible extensions

To resolve our own shortcomings, a possible extension lies within the stealing dynamic to implement a behaviour in which the thief is running away with the sphere, and only be able to “eat” it at its home. It would also be interesting to incorporate behaviors that depended on the other ePucks actions such as accelerating and avoiding obstacles when the robo-cop is perceived by the sensors. Additionally, adding more ePucks to accompany the thief and the police could create more of a storyline and act as rivaling colonies found in nature.

## REFERENCES

[1] Schmitz O. (2017). Predator and prey functional traits: understanding the adaptive machinery driving predator-prey interactions. *F1000Research, 6*, 1767. https://doi.org/10.12688/f1000research.11813.1

[2] Mahadevan, K., Somanath, S., & Sharlin, E. (2018, March). " Fight-or-Flight" Leveraging Instinctive Human Defensive Behaviors for Safe Human-Robot Interaction. In *Companion of the 2018 ACM/IEEE International Conference on Human-Robot Interaction* (pp. 183-184).

[3] Lan, G., Chen, J., & Eiben, A. E. (2019, July). Evolutionary predator-prey robot systems: From simulation to real world. In *Proceedings of the Genetic and Evolutionary Computation Conference Companion* (pp. 123-124).